In [1]:
from google.colab import drive

drive.mount("/content/gdrive")  
!pwd  # show current path 
%cd "/content/gdrive/MyDrive/Reto2/datasets"

Mounted at /content/gdrive
/content
/content/gdrive/MyDrive/Reto2/datasets


# Modelo de regresion logistica usando sklearn

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import explained_variance_score, mean_squared_error, confusion_matrix

In [39]:
dataset_full = pd.read_csv('trips2.csv')
comunas_origin = dataset_full['origin']
comunas_origin = pd.get_dummies(comunas_origin)
dataset_full = dataset_full.join(comunas_origin)
dataset_full = dataset_full.drop(['Unnamed: 0','lon','lat','origin'], axis=1)

In [ ]:
df_x = dataset_full.loc[:, dataset_full.columns != 'destination']

df_y = dataset_full["destination"]

 ########################### separación de los datos y entrenamiento del modelo ###############################
Xtrain, Xtest, ytrain, ytest = train_test_split(df_x.values, df_y.values,random_state=1)
model = LogisticRegression(fit_intercept = True)
model.fit(Xtrain,ytrain)
pred_y = model.predict(Xtest)
acc = accuracy_score(ytest, pred_y)

In [ ]:
########################## accuracy ########################
print("accuracy of the model: "+ str(acc))


accuracy of the model: 0.06548930373466055


# Modelo random forest Tensorflow

In [3]:
# Install TensorFlow Decision Forests
!pip install tensorflow_decision_forests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.2 MB 38.9 MB/s 
     |████████████████████████████████| 588.3 MB 7.4 kB/s 
     |████████████████████████████████| 439 kB 69.7 MB/s 
     |████████████████████████████████| 6.0 MB 51.5 MB/s 
     |████████████████████████████████| 1.7 MB 66.2 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 

In [4]:
columnas = ['destination', 'start_time', 'end_time', '0', 'ALHUÉ', 'BUIN',
       'CALERA DE TANGO', 'CERRILLOS', 'CERRO NAVIA', 'COLINA', 'CONCHALÍ',
       'CURACAVÍ', 'EL BOSQUE', 'EL MONTE', 'ESTACIÓN CENTRAL', 'HUECHURABA',
       'INDEPENDENCIA', 'ISLA DE MAIPO', 'LA CISTERNA', 'LA FLORIDA',
       'LA GRANJA', 'LA PINTANA', 'LA REINA', 'LAMPA', 'LAS CONDES',
       'LO BARNECHEA', 'LO ESPEJO', 'LO PRADO', 'MACUL', 'MAIPÚ',
       'MARÍA PINTO', 'MELIPILLA', 'PADRE HURTADO', 'PAINE',
       'PEDRO AGUIRRE CERDA', 'PEÑAFLOR', 'PEÑALOLÉN', 'PIRQUE', 'PROVIDENCIA',
       'PUDAHUEL', 'PUENTE ALTO', 'QUILICURA', 'QUINTA NORMAL', 'RECOLETA',
       'RENCA', 'SAN BERNARDO', 'SAN JOAQUÍN', 'SAN JOSÉ DE MAIPO',
       'SAN MIGUEL', 'SAN PEDRO', 'SAN RAMÓN', 'SANTIAGO', 'TALAGANTE',
       'TILTIL', 'VITACURA', 'ÑUÑOA']

In [8]:
import tensorflow_decision_forests as tfdf # Load TensorFlow Decision Forests

# Load the training dataset using pandas
import pandas as pd

dataset_full = pd.read_csv('trips2.csv')
dataset_full = dataset_full.sample(50000)
comunas_origin = dataset_full['origin']
comunas_origin = pd.get_dummies(comunas_origin)
dataset_full = dataset_full.join(comunas_origin)
dataset_full = dataset_full.drop(['Unnamed: 0','lon','lat','origin'], axis=1)

train_df,test_df = train_test_split(dataset_full.values,random_state=42)

#### como la funcion anterior nos devuelve un np array, lo regreso a datafram
train_df = pd.DataFrame(train_df)
test_df = pd.DataFrame(test_df)
## les devuelvo los nombres de las columnas (se perdieron al volverse np_array)
train_df.columns = columnas
test_df.columns = columnas


# Convert the pandas dataframe into a TensorFlow dataset
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label="destination")
  
# Train the model
model = tfdf.keras.RandomForestModel()
model.fit(train_ds)

Use /tmp/tmpbbe588ru as temporary training directory
Reading training dataset...
Training dataset read in 0:00:01.251060. Found 37500 examples.
Training model...
Model trained in 0:01:30.413910
Compiling model...
Model compiled.


In [9]:
# Load the testing dataset
# Convert it to a TensorFlow dataset
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_df, label="destination")

# Evaluate the model
model.compile(metrics=["accuracy",'sparse_categorical_crossentropy'])
print('accuracy_test: '+ str(model.evaluate(test_ds)))
print('accuracy_train: '+ str(model.evaluate(train_ds)))

13/13 [==============================] - 1s 87ms/step - loss: 0.0000e+00 - accuracy: 0.5868 - sparse_categorical_crossentropy: 4.2150
accuracy_test: [0.0, 0.5867999792098999, 4.215045928955078]
38/38 [==============================] - 4s 92ms/step - loss: 0.0000e+00 - accuracy: 0.5958 - sparse_categorical_crossentropy: 3.2725
accuracy_train: [0.0, 0.5958133339881897, 3.272456169128418]


In [ ]:
model.summary()

Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (55):
	0
	ALHUÉ
	BUIN
	CALERA_DE_TANGO
	CERRILLOS
	CERRO_NAVIA
	COLINA
	CONCHALÍ
	CURACAVÍ
	EL_BOSQUE
	EL_MONTE
	ESTACIÓN_CENTRAL
	HUECHURABA
	INDEPENDENCIA
	ISLA_DE_MAIPO
	LAMPA
	LAS_CONDES
	LA_CISTERNA
	LA_FLORIDA
	LA_GRANJA
	LA_PINTANA
	LA_REINA
	LO_BARNECHEA
	LO_ESPEJO
	LO_PRADO
	MACUL
	MAIPÚ
	MARÍA_PINTO
	MELIPILLA
	PADRE_HURTADO
	PAINE
	PEDRO_AGUIRRE_CERDA
	PEÑAFLOR
	PEÑALOLÉN
	PIRQUE
	PROVIDENCIA
	PUDAHUEL
	PUENTE_ALTO
	QUILICURA
	QUINTA_NORMAL
	RECOLETA
	RENCA
	SANTIAGO
	SAN_BERNARDO
	SAN_JOAQUÍN
	SAN_JOSÉ_DE_MAIPO
	SAN_MIGUEL
	SAN_PEDRO
	SAN_RAMÓN
	TALAGANTE
	TILTIL
	VITACURA
	end_time
	start_time
	ÑUÑOA

No weights

Var

In [ ]:
# Create another model with specified hyper-parameters
model2 = tfdf.keras.GradientBoostedTreesModel(
    num_trees=200,
    growing_strategy="BEST_FIRST_GLOBAL",
    max_depth=2,
    split_axis="SPARSE_OBLIQUE",
    )

# Evaluate the model
model2.compile(metrics=["accuracy"])
model2.fit(train_ds)
print(model2.evaluate(test_ds))
# >> 0.986851

Use /tmp/tmpdzf_n40l as temporary training directory
Reading training dataset...
Training dataset read in 0:00:01.874623. Found 37500 examples.
Training model...


# Modelo con una red Neuronal

In [10]:
import matplotlib.pyplot as plt
import numpy as np 

import tensorflow as tf
import pandas as pd

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPool2D

from sklearn.model_selection import train_test_split
 
import warnings
warnings.filterwarnings('ignore')

In [11]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

In [12]:
len(columnas) #destination = columnas[0]

56

In [13]:
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

In [17]:
df = pd.read_csv('trips2.csv')
df = df.sample(50000)
comunas_origin = df['origin']
comunas_origin = pd.get_dummies(comunas_origin)
df = df.join(comunas_origin)
df = df.drop(['Unnamed: 0','lon','lat','origin'], axis=1)

le = preprocessing.LabelEncoder()   
le.fit(df.destination)
df['destination'] = le.transform(df.destination)

df_x = df.loc[:, df.columns != 'destination']
df_y = df["destination"]

#df_x = df_x.values
#df_y = df_y.values

X_train, X_test, y_train, y_test = train_test_split(df_x, df_y)
#X_train=X_train.values.astype(np.float32)
#y_train=y_train.values.astype(np.float32)
#df_train = pd.DataFrame(X_train, columns=columnas)

comunas = encode_text_index(df,'destination')


model3 = Sequential()
model3.add(Dense(100, input_dim=55))
model3.add(Dense(100, activation="relu"))
model3.add(Dense(200, activation="relu"))
model3.add(Dense(200, activation="relu"))
model3.add(Dropout(0.25))
model3.add(Dense(200, activation="relu"))
model3.add(Dense(100, activation="relu"))
model3.add(Dense(53, activation="softmax"))

mae = tf.keras.metrics.MeanAbsoluteError()
model3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
               loss="sparse_categorical_crossentropy",                         
               metrics=["accuracy",mae])               

model3.summary()                                                  

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 100)               5600      
                                                                 
 dense_8 (Dense)             (None, 100)               10100     
                                                                 
 dense_9 (Dense)             (None, 200)               20200     
                                                                 
 dense_10 (Dense)            (None, 200)               40200     
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_11 (Dense)            (None, 200)               40200     
                                                                 
 dense_12 (Dense)            (None, 100)              

In [25]:
history = model3.fit(X_train, y_train,                                      
           validation_data = (X_test,y_test),
           epochs=30)                                                    


Epoch 1/30
1172/1172 [==============================] - 6s 5ms/step - loss: 3.6931 - accuracy: 0.0613 - mean_absolute_error: 28.4655 - val_loss: 3.6989 - val_accuracy: 0.0607 - val_mean_absolute_error: 28.4744
Epoch 2/30
1172/1172 [==============================] - 4s 4ms/step - loss: 3.6931 - accuracy: 0.0608 - mean_absolute_error: 28.4655 - val_loss: 3.6998 - val_accuracy: 0.0571 - val_mean_absolute_error: 28.4744
Epoch 3/30
1172/1172 [==============================] - 4s 4ms/step - loss: 3.6930 - accuracy: 0.0605 - mean_absolute_error: 28.4655 - val_loss: 3.6986 - val_accuracy: 0.0607 - val_mean_absolute_error: 28.4744
Epoch 4/30
1172/1172 [==============================] - 4s 4ms/step - loss: 3.6930 - accuracy: 0.0607 - mean_absolute_error: 28.4655 - val_loss: 3.6985 - val_accuracy: 0.0571 - val_mean_absolute_error: 28.4744
Epoch 5/30
1172/1172 [==============================] - 4s 4ms/step - loss: 3.6929 - accuracy: 0.0620 - mean_absolute_error: 28.4655 - val_loss: 3.6995 - val_ac

In [26]:
history.history.keys()

dict_keys(['loss', 'accuracy', 'mean_absolute_error', 'val_loss', 'val_accuracy', 'val_mean_absolute_error'])

In [27]:
acc = history.history['accuracy']
test_acc = history.history['val_accuracy']
loss = history.history['loss']
test_loss = history.history['val_loss']

print('train accuracy:'+ str(acc[-1]))
print('test accuracy:'+ str(test_acc[-1]))
print('train loss:'+ str(loss[-1]))
print('test loss:'+ str(test_loss[-1]))

train accuracy:0.06034666672348976
test accuracy:0.06072000041604042
train loss:3.6929545402526855
test loss:3.6995785236358643


## Modelo con XGboost

In [38]:
from sklearn.metrics import accuracy_score, f1_score,hinge_loss
import xgboost as xgb

In [39]:
df = pd.read_csv('trips2.csv')
df = df.sample(10000)
comunas_origin = df['origin']
comunas_origin = pd.get_dummies(comunas_origin)
df = df.join(comunas_origin)
df = df.drop(['Unnamed: 0','lon','lat','origin'], axis=1)

le = preprocessing.LabelEncoder()   
le.fit(df.destination)
df['destination'] = le.transform(df.destination)

df_x = df.loc[:, df.columns != 'destination']
df_y = df["destination"]

X_train, X_test, y_train, y_test = train_test_split(df_x, df_y)

In [40]:
from xgboost.sklearn import XGBClassifier
#initial model
xgb1 = XGBClassifier(learning_rate=0.1,
                    n_estimators=1000,
                    max_depth=5,
                    min_child_weight=1,
                    gamma=0,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    objective='multi:softmax',
                    nthread=4,
                    num_class=53,
                    seed=27)

In [43]:
training_data=X_train
epochs=300
sk_model = xgb1


sk_model.fit(X_train, y_train)

train = xgb.DMatrix(X_train, label=y_train)
test = xgb.DMatrix(X_test, label=y_test)
#val = xgb.DMatrix(X_train['X_val'], label=X_train['Y_val'])
params = sk_model.get_xgb_params()
metrics = ['mlogloss','merror']
params['eval_metric'] = metrics 
store = {}
evallist = [(train,'train'),(test, 'test')] #,
xgb_model = xgb.train(params, train, epochs, evallist,evals_result=store,verbose_eval=100)

print('XGBoost Accuracy: '+str(accuracy_score(sk_model.predict(X_test), y_test))) 
print('XGBoost Train Accuracy: '+str(sk_model.score(X_train,y_train))) 
print(store)

[0]	train-mlogloss:2.66064	train-merror:0.2384	test-mlogloss:2.67141	test-merror:0.2448
[100]	train-mlogloss:0.513079	train-merror:0.155067	test-mlogloss:1.17969	test-merror:0.2104
[200]	train-mlogloss:0.283274	train-merror:0.0764	test-mlogloss:1.25506	test-merror:0.2252
[299]	train-mlogloss:0.174772	train-merror:0.028133	test-mlogloss:1.3208	test-merror:0.2292
XGBoost Accuracy: 0.7536
XGBoost Train Accuracy: 1.0
{'train': {'mlogloss': [2.660635, 2.41732, 2.212889, 2.075457, 1.970945, 1.860269, 1.775924, 1.69813, 1.627998, 1.561403, 1.5013, 1.449788, 1.404234, 1.362313, 1.320475, 1.284765, 1.251265, 1.220174, 1.190277, 1.160585, 1.136143, 1.111307, 1.088294, 1.066206, 1.045482, 1.026604, 1.007576, 0.989337, 0.97281, 0.956253, 0.941859, 0.927813, 0.914149, 0.901149, 0.888891, 0.877752, 0.867489, 0.856627, 0.846519, 0.837332, 0.827148, 0.818503, 0.810086, 0.802095, 0.794082, 0.786334, 0.779105, 0.771336, 0.763808, 0.756355, 0.749388, 0.74261, 0.735399, 0.728798, 0.72219, 0.715992, 0.7096

In [42]:
print('XGBoost Train Accuracy: '+str(sk_model.score(X_train,y_train))) 

XGBoost Train Accuracy: 1.0
